<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as mp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [30]:
import warnings
warnings.simplefilter("ignore")

In [4]:
df = sns.load_dataset('tips')

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [7]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [8]:
df.duplicated().sum()

1

In [9]:
df.shape

(244, 7)

In [10]:
x = df.iloc[:,1:]

In [11]:
x.head()

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4


In [12]:
y = df.iloc[:,:1]

In [13]:
y.head()

,total_bill
0,16.99
1,10.34
2,21.01
3,23.68
4,24.59


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=10)

In [16]:
num_process = Pipeline(
steps=[
    ("imputaion_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
]) 

<IPython.core.display.Javascript object>

In [17]:
cat_process = Pipeline(
steps=[
    ("imputation_constant", SimpleImputer(fill_value="missing", strategy="constant")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

In [18]:
preprocess = ColumnTransformer([
    ("categorical", cat_process, ["sex","smoker","day","time"]),
    ("numerical", num_process, ["tip","size"])
])

In [19]:
pipe = Pipeline(
    steps=[
        ("preprocessor", preprocess),
        ("regressor", RandomForestRegressor())
    ]
)

In [20]:
from sklearn import set_config

In [21]:
set_config(display='diagram')

In [22]:
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['sex', 'smoker', 'day',
                                                   'time']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputaion_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['tip', 'size'])])),
                ('regressor', RandomForestRegressor())])

In [23]:
# param_grid = {
#     "regressor__n_estimators": [200,500],
#     "regressor__max_feature": ["auto","sqrt","log2"],
#     "regressor__max_depth": [4,5,6,7,8]
# }

In [26]:
# grid_search = GridSearchCV(pipe, param_grid=param_grid, n_jobs=5)

In [ ]:
# grid_search.fit(xtrain,ytrain)

In [31]:
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['sex', 'smoker', 'day',
                                                   'time']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputaion_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['tip', 'size'])])),
                ('regressor', RandomForestRegressor())])

In [32]:
ypred = pipe.predict(xtest)

In [33]:
print(ypred)

[19.167845   22.75046667 20.89269167 31.6815     23.01231667 22.1249
 19.6553     33.3518     18.4501     22.0701     14.8036     20.0076
  9.4819     30.6262     22.08446    10.12576667 12.63676417 11.28384167
 18.9118     21.63396667 14.1135     15.183325   14.40942    10.78195833
 38.4898     12.63676417 20.66236667 21.62366667 15.60205    33.9035
 32.2843     30.4709     13.72570238 13.867245   17.632375   23.45595
 19.1839     17.37791    20.8435     32.4318     21.99555333 28.1686
 10.34713333 32.6395     12.909465   14.12912    19.36717333 13.72570238
 14.32646857]
